# Lesson 4. SimPy &mdash; Monitors

### SA421 Fall 2015

## From last time... 

**Problem.** Customers visit the neighborhood hair stylist Fantastic Dan for haircuts. The customer interarrival time is exponentially distributed with a mean of 20 minutes. Each haircut takes Fantastic Dan anywhere from 15 to 25 minutes, uniformly distributed. This time also includes the initial greetings and the transaction of money at the end of the haircut.

1. Simulate 1 day of Dan’s operations. Assume Dan works continuously for 6 hours a day.
2. How many customers enter Dan's shop per day?
3. How many customers does Dan serve per day?
4. What is the time average number of customers in the queue? What is the maximum number of customers in the queue? 
5. What is the average time spent by a customer in the queue (i.e. average delay)? What is the maximum?

* First, some setup code. 
    * This is mostly the same from before. 
    * Observe that there are some new lines to make plotting easy to do in IPython Notebook.

In [ ]:
##### Setup #####
# Import everything from SimPy
from SimPy.Simulation import *

# Import seed initializer and random sampling functions from NumPy
from numpy.random import seed, uniform, exponential

# Import step plotting and histogram functions from Matplotlib
from matplotlib.pyplot import step, hist

# Run Matplotlib magic to show plots directly in the notebook
%matplotlib inline

# Make Matplotlib plots display as SVG files, which are cleaner
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

* Here is the simulation model for the Fantastic Dan problem from last time, modified to run for only 2 hours.

In [ ]:
##### Parameters #####
class P:
    # Customers arrive at the entrance with exponentially distributed
    # interarrival times with mean 20
    interarrivalTimeMean = 20
    
    # Service times are uniformly distributed between 15 and 25
    serviceTimeMin = 15
    serviceTimeMax = 25
    
    # One server: Fantastic Dan works by himself
    nServers = 1
    
    # Shop is open for 2 continuous hours
    simulationTimeMax =  2 * 60
    

##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives, joins queue
        print("Time {1}: {0} arrives and joins queue".format(self.name, now()))
        yield request, self, R.server
        
        # Customer is released from queue and starts service
        print("Time {1}: {0} begins service".format(self.name, now()))
        serviceTime = uniform(low = P.serviceTimeMin, high = P.serviceTimeMax)
        yield hold, self, serviceTime
        
        # Customer finishes service, leaves
        print("Time {1}: {0} ends service and leaves".format(self.name, now()))
        yield release, self, R.server

# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name="Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1

##### Resources #####
class R:
    # Server
    server = None


##### Model #####
def model():
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator (more on this later in the semester)
    seed(123)

    # Create the server resource
    R.server = Resource(capacity = P.nServers)

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

* Running the model, we see output similar to what we had last time.

In [ ]:
model()

* Based on our 1 simulation run, Dan cuts hair for 4 customers by the end of the day, and has 4 customers enter his shop.


* To compute the time average number of customers in the queue, we can use a monitor.

## What is a monitor?

* A **monitor** enables us to observe a variable of interest and hold a time series of its values.


* Monitoring certain aspects of a resource in SimPy is quite easy:
    - the number of entities in the resource's queue
    - the number of entities being processed by the resource
    

* Monitoring other quantities in SimPy requires a little more work, but is still pretty easy.

## Built-in monitors for resources

* We can activate the built-in monitors for a resource by adding the argument `monitored = True` when we call `Resource()` to create the resource.


* Doing this establishes two monitors:
    * **`waitMon`** to record changes in the queue.
    * **`actMon`** to record changes in entities being served.


* Let's modify `model()` from our Fantastic Dan simulation to activate these monitors for the server.

In [ ]:
##### Model #####
def model():
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator (more on this later in the semester)
    seed(123)

    # Create the server resource
    R.server = Resource(capacity = P.nServers)

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

* Now, let's run the model.

In [ ]:
##### Experiment #####
model()

* Everything looks the same as before, but...


* Now, we can access a resource's monitors using `.waitMon` and `.actMon`.


* For example, to examine `waitMon` for `R.server`, we can write:

In [ ]:
print("waitMon for R.server: {0}".format( ))

* As we can see, this monitor is a list of lists.


* Comparing `waitMon` with the output of the simulation, we see that the "inner lists" are observations of the form $[t, y]$, where

$$\begin{aligned}
t & = \text{time}\\
y & = \text{length of queue at time $t$}
\end{aligned}$$

* `actMon` works in a similar fashion:

In [ ]:
print("actMon for R.server: {0}".format( ))

* Now, `actMon` consists of data points of the form $[t, y]$, where

$$\begin{aligned}
t & = \text{time}\\
y & = \text{number of entities being processed by the resource at time $t$}
\end{aligned}$$

* To make these monitors easier to work with, we can split them into two lists:
    - One list consisting of the times $t$
    - Another list consisting of the values $y$


* For example, to get separate lists of the times and values of `waitMon`, we can write:

In [ ]:
print("waitMon times = {0}".format( ))
print("waitMon values = {0}".format( ))

* With these separate lists, we can plot the number of customers in the queue as a function of time using the `step` function from Matplotlib


* `step(t, y, where = "post")` creates a step plot: 
    - `t` is a list of the horizontal axis breakpoints, assumed to be in nondecreasing order
    - `y` is a list of the corresponding vertical axis values
    - `where = "post"` tells `step` to assign the interval between `t[i]` and `t[i+1]` a value of `y[i]`.
    - [Matplotlib documentation on `step` &mdash; scroll to find `matplotlib.pyplot.step`](http://matplotlib.org/api/pyplot_api.html)

* We can also easily get some summary statistics from these monitors, such as 
    * the time average number of customers in the queue, and 
    * the maximum number of customers in the queue.

In [ ]:
# Get average number of customers waiting for a haircut
print("Time average number of customers in queue = {0}".format( ))

# Get maximum number of customers waiting for a haircut
print("Maximum number of customers in queue = {0}".format( )))

* Other summary statistics of interest for a monitor named `monitor`:
    - `monitor.count()` gives the current number of observations
    - `monitor.total()` gives the sum of the $y$ values
    - `monitor.mean()` gives the average of the $y$ values, <span style="color:#a00000">ignoring the corresponding times of the $y$ values</span>
    - `monitor.var()` gives the sample variance of the y values, <span style="color:#a00000">ignoring the corresponding times of the $y$ values</span>
    
    
* [Here](http://simpy.sourceforge.net/old/SimPy_Manual/Manuals/Manual.html#data-summaries) is the SimPy documentation on summary statistics available for a monitor.

## Custom-made monitors

* To figure out how long a customer waits in the queue on average (i.e. average delay), we need to monitor the customer behavior directly.


* This can be accomplished using a custom-made monitor.


* Like with parameters and resources, we define variables for monitors in a common class.

In [ ]:
##### Monitors #####
class M:
    delay = None

* Also like with resources, we create the monitor after we initialize the simulation.

In [ ]:
##### Model #####
def model():
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator (more on this later in the semester)
    seed(123)

    # Create the server resource
    R.server = Resource(capacity = P.nServers, monitored = True)

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

* We need to modify the behavior of the `Customer` process to monitor the delay (i.e. time in queue) of each customer.


* For a monitor named `monitor`, we use `monitor.observe(value)` to record `value` at the current time of the simulation.

In [ ]:
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives, joins queue
        print("Time {1}: {0} arrives and joins queue".format(self.name, now()))
        yield request, self, R.server
        
        # Customer is released from queue and starts service
        print("Time {1}: {0} begins service".format(self.name, now()))
        serviceTime = uniform(low = P.serviceTimeMin, high = P.serviceTimeMax)
        yield hold, self, serviceTime
        
        # Customer finishes service, leaves
        print("Time {1}: {0} ends service and leaves".format(self.name, now()))
        yield release, self, R.server

* Now, we can run the model again, with the delay monitor.

In [ ]:
##### Experiment #####
model()

* We can examine the delay of each customer that we recorded using this monitor:

In [ ]:
# Print customer delay and the time of recording
print("Time recorded = {0}".format( ))
print("Delay = {0}".format( ))

* Finally, we can compute the average and maximum delay:

In [ ]:
# Get average delay
print("Average delay = {0}".format( ))

# Get maximum delay
print("Maximum delay = {0}".format( )))

* While we're at it, we can plot a histogram of the customer delay using the `hist` function from Matplotlib.


* `hist(x, bins)` creates a histogram
    - `x` is a list of data points
    - `bins` is a list of the edges of the bin intervals
    - Without any other arguments, `hist` automatically creates bins for the histogram
    - [Here](http://matplotlib.org/api/pyplot_api.html) is the Matplotlib documentation for `hist` &ndash; scroll down to find `matplotlib.pyplot.hist`

In [ ]:
# Define edges of the histogram bin intervals

# Get histogram of customer time in queue


## On your own...

* First, change the simulation to run for 6 hours instead of 2.


* Since we defined the class `P` above, we do not need to redefine the entire class `P` &ndash; we can just redefine `P.simulationTimeMax` by itself:

In [ ]:
P.simulationTimeMax = 6 * 60

* In the Fantastic Dan problem, since there is only 1 server, the time average number of busy servers gives you the server utilization: the fraction of time the server is busy.


* Given what we learned today, how can you compute the time average number of busy servers?

* Create a monitor that records a value for each customer that enters the shop: 
    - 1 if a customer waits in the queue for more than 15 minutes, 
    - 0 otherwise.


* Using this monitor, find the fraction of customers that wait in the queue for more than 15 minutes.

In [ ]:
# Define dummy variable for a new monitor


In [ ]:
# Modify Customer process to record whether a customer
# is in the queue for more than 15 minutes


In [ ]:
# Modify model to create new monitor


In [ ]:
# Run new model


In [ ]:
# Find the fraction of customers that wait in the queue for more than 15 minutes
